In [18]:
import os
import sys

# 打印当前工作目录
print("Current working directory:", os.getcwd())

# 更改当前工作目录（如果必要）
new_dir = '/root/autodl-tmp/'
os.chdir(new_dir)
print("Changed working directory to:", os.getcwd())

# 确保当前目录在系统路径中
if new_dir not in sys.path:
    sys.path.append(new_dir)

# 清除之前的导入缓存
if 'myReader' in sys.modules:
    del sys.modules['myReader']

# 尝试导入 convert_example 函数
from myReader import get_dataLoader

# # 测试函数
# convert_example()

Current working directory: /root/autodl-tmp
Changed working directory to: /root/autodl-tmp


In [1]:
# !/usr/bin/env python3
"""
==== No Bugs in code, just some Random Unexpected FEATURES ====
┌─────────────────────────────────────────────────────────────┐
│┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐│
││Esc│!1 │@2 │#3 │$4 │%5 │^6 │&7 │*8 │(9 │)0 │_- │+= │|\ │`~ ││
│├───┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴───┤│
││ Tab │ Q │ W │ E │ R │ T │ Y │ U │ I │ O │ P │{[ │}] │ BS  ││
│├─────┴┬──┴┬──┴┬──┴┬──┴┬──┴┬──┴┬──┴┬──┴┬──┴┬──┴┬──┴┬──┴─────┤│
││ Ctrl │ A │ S │ D │ F │ G │ H │ J │ K │ L │: ;│" '│ Enter  ││
│├──────┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴─┬─┴────┬───┤│
││ Shift  │ Z │ X │ C │ V │ B │ N │ M │< ,│> .│? /│Shift │Fn ││
│└─────┬──┴┬──┴──┬┴───┴───┴───┴───┴───┴──┬┴───┴┬──┴┬─────┴───┘│
│      │Fn │ Alt │         Space         │ Alt │Win│   HHKB   │
│      └───┴─────┴───────────────────────┴─────┴───┘          │
└─────────────────────────────────────────────────────────────┘

测试训练好的模型效果。

Author: pankeyu
Date: 2023/01/05
"""
from rich import print
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM
import json

device = 'cuda:0'
max_source_seq_len = 256
model_checkpoint = 'model_best2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint) 

model.to(device).eval()
import torch

def inference(question: str, context: str, answer: str):
    """
    inference函数。

    Args:
        qustion (str): 问题
        context (str): 原文
    """
    input_seq = "给定上下文：{} 。回答问题：{}".format(context,question)
    inputs = tokenizer(
        text=input_seq,
        truncation=True,
        max_length=max_source_seq_len,
        padding=True,
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            max_length=64,
            num_beams=8,
            no_repeat_ngram_size=2,
            eos_token_id=tokenizer.eos_token_id,
        ).cpu().numpy()
    output = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(" ", "")
    print(f'Q: "{question}"')
    print(f'C: "{context}"')
    print(f'A: "{output}"')

if __name__ == '__main__':
    # question = '治疗宫颈糜烂的最佳时间'
    # context = '专家指出，宫颈糜烂治疗时间应选在月经干净后3-7日，因为治疗之后宫颈有一定的创面，如赶上月经期易发生感染。因此患者应在月经干净后3天尽快来医院治疗。同时应该注意，术前3天禁同房，有生殖道急性炎症者应治好后才可进行。'
    # inference(qustion=question, context=context)
    data_file = "data/DuReaderQG/dev.json"
    with open(data_file, 'rt', encoding='utf-8') as f:
        for idx, line in enumerate(f):
            one_data = json.loads(line.strip())
            inference(question=one_data['question'], context=one_data['context'], answer=one_data['answer'])

                

T5ForConditionalGeneration(
  (shared): Embedding(21228, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(21228, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro